<a href="https://colab.research.google.com/github/lucacamus13/dashboard-ARG/blob/main/inflaci%C3%B3n_emae_ARG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 42.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import requests
from io import BytesIO
from datetime import datetime

# Inflación

In [15]:
import pandas as pd
import requests
from io import BytesIO
from datetime import datetime
import calendar

def obtener_datos_indec_completo():
    """
    Lee las 3 hojas principales del Excel del INDEC:
    - Hoja 0: Variación mensual
    - Hoja 1: Variación interanual
    - Hoja 2: Índices IPC
    """
    # Calcular el mes actual
    hoy = datetime.now()
    mes_actual_str = str(hoy.month).zfill(2) # Obtener el número del mes actual y formatear con cero inicial si es necesario

    # Construir el URL con el mes actual
    url = f"https://www.indec.gob.ar/ftp/cuadros/economia/sh_ipc_{mes_actual_str}_25.xls"


    try:
        print(f"Descargando Excel desde: {url}")

        response = requests.get(url, timeout=30)
        response.raise_for_status()

        print(f"✓ Archivo descargado: {len(response.content)} bytes")

        # Leer las 3 hojas principales
        hojas_datos = {}
        nombres_hojas = [
            'Variación mensual IPC Nacional',
            'Var. interanual IPC Nacional',
            'Índices IPC Cobertura Nacional'
        ]

        for i, nombre in enumerate(nombres_hojas):
            try:
                df = pd.read_excel(BytesIO(response.content), sheet_name=i, engine='xlrd')
                hojas_datos[nombre] = df
                print(f"✓ Hoja {i} '{nombre}': {len(df)} filas x {len(df.columns)} columnas")
            except Exception as e:
                print(f"❌ Error leyendo hoja {i}: {e}")

        return hojas_datos

    except Exception as e:
        print(f"❌ Error: {e}")
        return {}

def extraer_nivel_general_de_hoja(df, nombre_hoja):
    """
    Extrae datos de 'Nivel general' de cualquier hoja
    """
    if df.empty:
        return pd.DataFrame()

    print(f"\n=== PROCESANDO {nombre_hoja.upper()} ===")

    # Buscar fila con fechas
    fila_fechas = None
    for i, fila in df.iterrows():
        # Check if the row contains datetime objects in columns from the second one onwards
        fechas_count = sum(1 for val in fila.iloc[1:] if isinstance(val, datetime))
        if fechas_count > 5: # Reduced the threshold slightly
            fila_fechas = i
            break

    # Buscar fila "Nivel general"
    fila_nivel_general = None
    for i, fila in df.iterrows():
        if pd.notna(fila.iloc[0]) and str(fila.iloc[0]).strip() == 'Nivel general':
            fila_nivel_general = i
            break

    if fila_fechas is None or fila_nivel_general is None:
        print(f"❌ No se encontraron datos en {nombre_hoja}")
        return pd.DataFrame()

    print(f"✓ Fechas en fila {fila_fechas}, Nivel general en fila {fila_nivel_general}")

    # Extraer fechas y valores
    fechas = df.iloc[fila_fechas].values[1:]  # Saltar primera columna
    valores = df.iloc[fila_nivel_general].values[1:]  # Saltar primera columna

    # Crear DataFrame
    datos = []
    # Iterate through dates and values, including the last available data point
    for fecha, valor in zip(fechas, valores):
        if pd.notna(fecha) and isinstance(fecha, datetime):
            # Check if the value is not NaN before appending
            if pd.notna(valor):
                try:
                    datos.append({
                        'fecha': fecha,
                        'valor': float(valor)
                    })
                except ValueError:
                    print(f"⚠️ Advertencia: No se pudo convertir el valor '{valor}' a float para la fecha {fecha}. Se omitirá este punto de datos.")
                    pass # Skip this data point if conversion fails
            else:
                 print(f"⚠️ Advertencia: Valor NaN encontrado para la fecha {fecha}. Se omitirá este punto de datos.")


    if datos:
        df_resultado = pd.DataFrame(datos).sort_values('fecha').reset_index(drop=True)
        print(f"✓ {len(df_resultado)} registros extraídos")
        return df_resultado

    return pd.DataFrame()


def combinar_datos_inflacion(hojas_datos):
    """
    Combina datos de las 3 hojas en un DataFrame completo
    """
    print("\n=== COMBINANDO DATOS DE LAS 3 HOJAS ===")

    # Extraer datos de cada hoja
    datos_combinados = {}

    # Hoja 0: Variación mensual
    if 'Variación mensual IPC Nacional' in hojas_datos:
        df_mensual = extraer_nivel_general_de_hoja(
            hojas_datos['Variación mensual IPC Nacional'],
            'Variación mensual'
        )
        if not df_mensual.empty:
            datos_combinados['variacion_mensual'] = df_mensual.set_index('fecha')['valor']

    # Hoja 1: Variación interanual
    if 'Var. interanual IPC Nacional' in hojas_datos:
        df_interanual = extraer_nivel_general_de_hoja(
            hojas_datos['Var. interanual IPC Nacional'],
            'Variación interanual'
        )
        if not df_interanual.empty:
            datos_combinados['variacion_interanual'] = df_interanual.set_index('fecha')['valor']

    # Hoja 2: Índices IPC
    if 'Índices IPC Cobertura Nacional' in hojas_datos:
        df_indices = extraer_nivel_general_de_hoja(
            hojas_datos['Índices IPC Cobertura Nacional'],
            'Índices IPC'
        )
        if not df_indices.empty:
            datos_combinados['ipc_indice'] = df_indices.set_index('fecha')['valor']

    if not datos_combinados:
        print("❌ No se pudieron extraer datos de ninguna hoja")
        return pd.DataFrame()

    # Combinar todos los datos
    df_final = pd.DataFrame(datos_combinados)
    df_final = df_final.reset_index()

    # Rellenar valores faltantes si es necesario (considerando que el último mes puede tener solo un dato)
    # df_final = df_final.dropna(subset=['fecha']) # Remove this line to keep rows with only date

    # Ensure 'fecha' is datetime and sort
    df_final['fecha'] = pd.to_datetime(df_final['fecha'])
    df_final = df_final.sort_values('fecha').reset_index(drop=True)

    print(f"✓ DataFrame final: {len(df_final)} registros")
    print(f"✓ Columnas: {list(df_final.columns)}")

    return df_final


def actualizar_inflacion_completa():
    """
    Función principal que obtiene datos completos de inflación
    """
    print("=== OBTENIENDO DATOS COMPLETOS DE INFLACIÓN DEL INDEC ===")

    # Descargar y leer todas las hojas
    hojas_datos = obtener_datos_indec_completo()

    if not hojas_datos:
        return None

    # Combinar datos de las 3 hojas
    df_completo = combinar_datos_inflacion(hojas_datos)

    if df_completo.empty:
        print("❌ No se pudieron procesar los datos")
        return None

    # Guardar datos
    df_completo.to_csv("inflacion_argentina_completa.csv", index=False)
    print(f"\n✅ ÉXITO: Datos completos guardados en 'inflacion_argentina_completa.csv'")

    # Mostrar resumen
    mostrar_resumen_completo(df_completo)

    return df_completo

def mostrar_resumen_completo(df):
    """
    Mostrar resumen completo de los datos
    """
    if df is None or df.empty:
        return

    print(f"\n=== RESUMEN COMPLETO DE INFLACIÓN ===")
    print(f"Total registros: {len(df)}")
    print(f"Período: {df['fecha'].min().strftime('%Y-%m')} a {df['fecha'].max().strftime('%Y-%m')}")
    print(f"Columnas disponibles: {list(df.columns)}")

    # Mostrar datos de las columnas disponibles
    columnas_numericas = [col for col in df.columns if col != 'fecha']

    for col in columnas_numericas:
        if col in df.columns and df[col].notna().any():
            print(f"\n{col.upper()}:")
            print(f"  Promedio: {df[col].mean():.2f}")
            print(f"  Máximo: {df[col].max():.2f}")
            print(f"  Mínimo: {df[col].min():.2f}")

    # Últimos 6 meses
    print(f"\n=== ÚLTIMOS 6 MESES ===")
    ultimos = df.tail(6)

    for _, row in ultimos.iterrows():
        fecha = row['fecha'].strftime('%Y-%m')
        linea = f"{fecha}:"

        if 'variacion_mensual' in df.columns and pd.notna(row.get('variacion_mensual')):
            linea += f" {row['variacion_mensual']:+.1f}% mensual"

        if 'variacion_interanual' in df.columns and pd.notna(row.get('variacion_interanual')):
            linea += f" | {row['variacion_interanual']:+.1f}% anual"

        if 'ipc_indice' in df.columns and pd.notna(row.get('ipc_indice')):
            linea += f" | IPC: {row['ipc_indice']:.1f}"

        print(linea)


def obtener_solo_variacion_mensual():
    """
    Función simple para obtener solo variaciones mensuales (más rápida)
    """
    hojas_datos = obtener_datos_indec_completo()

    if 'Variación mensual IPC Nacional' in hojas_datos:
        df_mensual = extraer_nivel_general_de_hoja(
            hojas_datos['Variación mensual IPC Nacional'],
            'Variación mensual'
        )
        if not df_mensual.empty:
            df_mensual.columns = ['fecha', 'variacion_mensual']
            return df_mensual

    return pd.DataFrame()

def obtener_solo_indices():
    """
    Función simple para obtener solo índices IPC
    """
    hojas_datos = obtener_datos_indec_completo()

    if 'Índices IPC Cobertura Nacional' in hojas_datos:
        df_indices = extraer_nivel_general_de_hoja(
            hojas_datos['Índices IPC Cobertura Nacional'],
            'Índices IPC'
        )
        if not df_indices.empty:
            df_indices.columns = ['fecha', 'ipc_indice']
            return df_indices

    return pd.DataFrame()

# Ejecutar
if __name__ == "__main__":
    # Opción completa (todas las hojas)
    df_completo = actualizar_inflacion_completa()

    if df_completo is not None:
        print(f"\n✅ DATOS DISPONIBLES:")
        print("  DataFrame completo: inflacion_argentina_completa.csv")
        print("\n🔧 USO EN CÓDIGO:")
        print("  # Datos completos")
        print("  df = pd.read_csv('inflacion_argentina_completa.csv')")
        print("\n  # O usar funciones específicas:")
        print("  from indec_scraper import obtener_solo_variacion_mensual")
        print("  df_mensual = obtener_solo_variacion_mensual()")

=== OBTENIENDO DATOS COMPLETOS DE INFLACIÓN DEL INDEC ===
Descargando Excel desde: https://www.indec.gob.ar/ftp/cuadros/economia/sh_ipc_09_25.xls
✓ Archivo descargado: 2286592 bytes
✓ Hoja 0 'Variación mensual IPC Nacional': 215 filas x 105 columnas
✓ Hoja 1 'Var. interanual IPC Nacional': 215 filas x 94 columnas
✓ Hoja 2 'Índices IPC Cobertura Nacional': 215 filas x 106 columnas

=== COMBINANDO DATOS DE LAS 3 HOJAS ===

=== PROCESANDO VARIACIÓN MENSUAL ===
✓ Fechas en fila 4, Nivel general en fila 8
✓ 104 registros extraídos

=== PROCESANDO VARIACIÓN INTERANUAL ===
✓ Fechas en fila 4, Nivel general en fila 8
✓ 93 registros extraídos

=== PROCESANDO ÍNDICES IPC ===
✓ Fechas en fila 4, Nivel general en fila 8
✓ 105 registros extraídos
✓ DataFrame final: 105 registros
✓ Columnas: ['fecha', 'variacion_mensual', 'variacion_interanual', 'ipc_indice']

✅ ÉXITO: Datos completos guardados en 'inflacion_argentina_completa.csv'

=== RESUMEN COMPLETO DE INFLACIÓN ===
Total registros: 105
Período:

In [16]:
import plotly.graph_objects as go
import pandas as pd

# Asegurarse de que la columna 'fecha' sea de tipo datetime
df_completo['fecha'] = pd.to_datetime(df_completo['fecha'])

# Crear el gráfico de Plotly
fig = go.Figure()

# Agregar la serie de Variación Interanual (línea) - Cambiado a naranja y con zorder
fig.add_trace(go.Scatter(
    x=df_completo['fecha'],
    y=df_completo['variacion_interanual'],
    mode='lines+markers',
    name='Variación Interanual',
    yaxis='y1', # Asociar al eje Y primario
    line=dict(color='orange'), # Color de la línea naranja
    marker=dict(color='orange'), # Color de los marcadores naranja
    zorder=2 # Asegurar que la línea esté adelante (zorder mayor)
))

# Agregar la serie de Variación Mensual (barras) - Cambiado a azul y con zorder
fig.add_trace(go.Bar(
    x=df_completo['fecha'],
    y=df_completo['variacion_mensual'],
    name='Variación Mensual',
    yaxis='y2', # Asociar al eje Y secundario
    marker=dict(color='blue', opacity=0.7), # Color de las barras azules con opacidad
    zorder=1 # Asegurar que las barras estén atrás (zorder menor)
))

# Configurar los ejes y el diseño - Colores de texto en negro y eliminar grilla
fig.update_layout(
    title=dict(
        text='Variación de la Inflación en Argentina (Nivel General) - Interactivo',
        x=0.5,  # Centrar el título (0.5 es el centro horizontal)
        xanchor='center' # Anclar el centro del título al centro horizontal
    ),
    xaxis=dict(
        title='Fecha',
        titlefont=dict(color='black'), # Título del eje X en negro
        tickfont=dict(color='black'), # Etiquetas del eje X en negro
        showgrid=False # Eliminar grilla del eje X
    ),
    yaxis=dict(
        title='Variación Interanual (%)',
        titlefont=dict(color='black'), # Título del eje Y1 en negro
        tickfont=dict(color='black'), # Etiquetas del eje Y1 en negro
        showgrid=False # Eliminar grilla del eje Y1
    ),
    yaxis2=dict(
        title='Variación Mensual (%)',
        titlefont=dict(color='black'), # Título del eje Y2 en negro
        tickfont=dict(color='black'), # Etiquetas del eje Y2 en negro
        overlaying='y', # Superponer al eje Y primario
        side='right',    # Colocar en el lado derecho
        showgrid=False # Eliminar grilla del eje Y2
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    hovermode='x unified', # Mostrar tooltip unificado al pasar el mouse
    plot_bgcolor='white', # Establecer el color de fondo del gráfico a blanco
    font=dict(family='Arial') # Establecer la fuente a Arial
)

# Mostrar el gráfico interactivo
fig.show()

# Tipo de Cambio

In [5]:
import yfinance as yf
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime, timedelta

# Obtener datos del dólar blue en Argentina (ARS=X es el ticker para USD/ARS)
dolar_argentina = yf.Ticker("ARS=X")

# Obtener datos históricos de los últimos 30 días
historial = dolar_argentina.history(period="1y") # Changed period to "1y" for one year

# Asegurarse de que el índice sea de tipo datetime y renombrarlo
historial.index = pd.to_datetime(historial.index)
historial = historial.rename_axis('fecha')

# Crear el gráfico de Plotly
fig = go.Figure()

# Agregar la serie de cotización (barras)
fig.add_trace(go.Scatter( # Changed to go.Scatter
    x=historial.index,
    y=historial['Close'],
    mode='lines', # Changed mode to 'lines'
    name='Cotización del Dólar',
    line=dict(color='blue') # Set line color to blue
))

# Configurar los ejes y el diseño
fig.update_layout(
    title=dict(
        text='Cotización del Dólar Estadounidense frente al Peso Argentino (USD/ARS)\nÚltimo Año - Interactivo', # Updated title
        x=0.5,  # Centrar el título
        xanchor='center' # Anclar el centro del título
    ),
    xaxis=dict(
        title='Fecha',
        titlefont=dict(color='black'), # Título del eje X en negro
        tickfont=dict(color='black'), # Etiquetas del eje X en negro
        showgrid=False # Eliminar grilla del eje X
    ),
    yaxis=dict(
        title='Pesos Argentinos por 1 USD',
        titlefont=dict(color='black'), # Título del eje Y en negro
        tickfont=dict(color='black'), # Etiquetas del eje Y en negro
        showgrid=False # Eliminar grilla del eje Y
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    hovermode='x unified', # Mostrar tooltip unificado
    plot_bgcolor='white', # Fondo blanco
    font=dict(family='Arial') # Fuente Arial
)

# Mostrar el gráfico interactivo
fig.show()

# Mostrar información adicional
if not historial.empty:
    ultimo_valor = historial['Close'].iloc[-1]
    ultima_fecha = historial.index[-1]
    print(f"Valor actual: ${ultimo_valor:.2f} ARS por 1 USD")
    print(f"Fecha de la última actualización: {ultima_fecha.strftime('%d-%m-%Y')}")

    if len(historial) > 1:
        variacion_porcentaje = ((ultimo_valor / historial['Close'].iloc[0]) - 1) * 100
        print(f"Variación en el período: {variacion_porcentaje:.2f}%")
    else:
        print("No hay suficientes datos para calcular la variación en el período.")
else:
    print("No se pudieron obtener datos del tipo de cambio.")

Valor actual: $1465.00 ARS por 1 USD
Fecha de la última actualización: 16-09-2025
Variación en el período: 52.72%


# Actividad

In [6]:
import pandas as pd
import requests
from io import BytesIO
from datetime import datetime
import numpy as np

def descargar_y_limpiar_emae():
    """
    Descarga y limpia los datos del EMAE desde el sitio del INDEC
    Estructura conocida: encabezados en fila 3, años en columna A, meses en columna B
    """
    url = "https://www.indec.gob.ar/ftp/cuadros/economia/sh_emae_mensual_base2004.xls"

    try:
        print("Descargando datos del EMAE...")
        response = requests.get(url, timeout=30)
        response.raise_for_status()

        print("Procesando archivo Excel...")
        # Leer el archivo Excel sin encabezados
        excel_data = pd.read_excel(BytesIO(response.content), engine='xlrd', header=None)

        print(f"Dimensiones del archivo: {excel_data.shape}")

        # Los encabezados están en la fila 3 (índice 2)
        encabezados_row = 2
        print(f"\nEncabezados en fila {encabezados_row + 1}:")
        for col in range(min(8, len(excel_data.columns))):
            val = excel_data.iloc[encabezados_row, col]
            if pd.notna(val):
                print(f"  Columna {col}: {val}")

        # Procesar los datos desde la fila 6 (índice 5)
        datos_procesados = []

        # Mapeo de nombres de meses
        meses = {
            'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4,
            'mayo': 5, 'junio': 6, 'julio': 7, 'agosto': 8,
            'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
        }

        print(f"\nProcesando datos desde la fila 6...")

        año_actual = None # Initialize year

        # Start from row 6 (index 5) as data starts from there
        for idx in range(5, len(excel_data)):
            row = excel_data.iloc[idx]

            # Columna A (índice 0): Año (may appear only on the first row of a year)
            # Columna B (índice 1): Mes
            año_val = row.iloc[0]
            mes_val = row.iloc[1]

            # Update current year if available
            if pd.notna(año_val):
                año_str = str(año_val).strip()
                if año_str.isdigit() and len(año_str) == 4:
                    año_actual = int(año_str)
                else:
                    año_actual = None # Reset if not a valid year format

            # Process only if we have a valid year and month
            if año_actual is not None and pd.notna(mes_val):
                mes_nombre = str(mes_val).strip().lower()
                if mes_nombre in meses:
                    mes_num = meses[mes_nombre]

                    # Filter from February 2004
                    if año_actual < 2004 or (año_actual == 2004 and mes_num < 2):
                        continue

                    # Format date as 'dd/mm/aaaa'
                    fecha = f"01/{mes_num:02d}/{año_actual}"

                    # Extraer las 6 columnas de datos (columnas C a H, índices 2 a 7)
                    valores = []
                    for col_idx in range(2, 8):  # Columnas C a H
                        try:
                            if col_idx < len(row):
                                val = row.iloc[col_idx]
                                if pd.notna(val):
                                    # Convertir a float, manejando diferentes formatos
                                    val_str = str(val).replace(',', '.')
                                    # Remover espacios y caracteres no numéricos (excepto punto, signo menos)
                                    val_clean = ''.join(c for c in val_str if c.isdigit() or c in '.-')
                                    if val_clean and val_clean.replace('.', '').replace('-', '').isdigit():
                                        valores.append(float(val_clean))
                                    else:
                                        valores.append(np.nan)
                                else:
                                    valores.append(np.nan)
                            else:
                                valores.append(np.nan)
                        except Exception as e:
                            print(f"Error procesando valor en fila {idx}, columna {col_idx}: {e}")
                            valores.append(np.nan)

                    # Crear registro
                    registro = {
                        'fecha': fecha,
                        'indice_serie_original': valores[0] if len(valores) > 0 else np.nan,
                        'var_respecto_año_anterior': valores[1] if len(valores) > 1 else np.nan,
                        'indice_desestacionalizado': valores[2] if len(valores) > 2 else np.nan,
                        'var_respecto_mes_anterior': valores[3] if len(valores) > 3 else np.nan,
                        'indice_tendencia_ciclo': valores[4] if len(valores) > 4 else np.nan,
                        'var_tendencia_mes_anterior': valores[5] if len(valores) > 5 else np.nan
                    }

                    datos_procesados.append(registro)

                    # Mostrar algunos registros para verification
                    if len(datos_procesados) <= 5 or len(datos_procesados) % 50 == 0:
                         print(f"  {fecha}: {año_actual}-{mes_nombre} -> {[round(v, 1) if not pd.isna(v) else 'NaN' for v in valores[:3]]}")


        # Crear DataFrame final
        if datos_procesados:
            df_limpio = pd.DataFrame(datos_procesados)

            # Convert 'fecha' to datetime for sorting
            df_limpio['fecha_sort'] = pd.to_datetime(df_limpio['fecha'], format='%d/%m/%Y')
            df_limpio = df_limpio.sort_values('fecha_sort')
            df_limpio = df_limpio.drop('fecha_sort', axis=1)

            # Resetear índice
            df_limpio = df_limpio.reset_index(drop=True)

            print(f"\nDataFrame creado exitosamente!")
            print(f"Número de registros: {len(df_limpio)}")
            print(f"Período: {df_limpio['fecha'].iloc[0]} a {df_limpio['fecha'].iloc[-1]}")

            # Verificar that we start from February 2004
            primera_fecha = pd.to_datetime(df_limpio['fecha'].iloc[0], format='%d/%m/%Y')
            if primera_fecha.year == 2004 and primera_fecha.month == 2:
                print("✓ Confirmado: Los datos comienzan en febrero de 2004")
            else:
                print(f"⚠️ Atención: Los datos comienzan en {primera_fecha.strftime('%B %Y')}")


            return df_limpio
        else:
            print("No se pudieron procesar los datos")
            return None

    except Exception as e:
        print(f"Error al descargar o procesar los datos: {e}")
        import traceback
        traceback.print_exc()
        return None

def mostrar_info_dataframe(df):
    """
    Muestra información detallada del DataFrame
    """
    if df is not None:
        print("\n" + "="*60)
        print("INFORMACIÓN DEL DATAFRAME")
        print("="*60)
        print(f"Shape: {df.shape}")
        print(f"\nColumnas:")
        for col in df.columns:
            print(f"  - {col}")

        print(f"\nPrimeras 5 filas:")
        print(df.head())

        print(f"\nÚltimas 5 filas:")
        print(df.tail())

        print(f"\nInformación de tipos de datos:")
        print(df.info())

        print(f"\nEstadísticas descriptivas:")
        print(df.describe())

# Ejecutar el proceso
if __name__ == "__main__":
    # Descargar y limpiar los datos
    df_emae = descargar_y_limpiar_emae()

    if df_emae is not None:
        # Mostrar información del DataFrame
        mostrar_info_dataframe(df_emae)

        # Guardar como CSV
        nombre_archivo = f"emae_datos_limpios_{datetime.now().strftime('%Y%m%d')}.csv"
        df_emae.to_csv(nombre_archivo, index=False, encoding='utf-8')
        print(f"\nArchivo guardado como: {nombre_archivo}")

        # Guardar como Excel
        nombre_archivo_excel = f"emae_datos_limpios_{datetime.now().strftime('%Y%m%d')}.xlsx"
        df_emae.to_excel(nombre_archivo_excel, index=False)
        print(f"Archivo guardado como: {nombre_archivo_excel}")
    else:
        print("No se pudo crear el DataFrame")

Descargando datos del EMAE...
Procesando archivo Excel...
Dimensiones del archivo: (265, 8)

Encabezados en fila 3:
  Columna 0: Período
  Columna 2: Índice Serie Original
2004=100
  Columna 3: Var % respecto a igual período del año anterior
  Columna 4: Índice Serie Desestacionalizada
2004=100
  Columna 5: Var % respecto al mes anterior
  Columna 6: Índice Serie Tendencia-Ciclo
2004=100
  Columna 7: Var % respecto al mes anterior

Procesando datos desde la fila 6...
  01/02/2004: 2004-febrero -> [90.2, 0.0, 98.2]
  01/03/2004: 2004-marzo -> [101.9, 0.0, 97.8]
  01/04/2004: 2004-abril -> [102.6, 0.0, 95.1]
  01/05/2004: 2004-mayo -> [109.9, 0.0, 96.9]
  01/06/2004: 2004-junio -> [105.7, 0.0, 98.7]
  01/03/2008: 2008-marzo -> [132.3, 2.4, 133.3]
  01/05/2012: 2012-mayo -> [157.4, -5.3, 140.2]
  01/07/2016: 2016-julio -> [143.7, -5.3, 144.4]
  01/09/2020: 2020-septiembre -> [127.2, -5.7, 130.7]
  01/11/2024: 2024-noviembre -> [148.1, 1.2, 150.6]

DataFrame creado exitosamente!
Número de 

In [7]:
df_emae

,fecha,indice_serie_original,var_respecto_año_anterior,indice_desestacionalizado,var_respecto_mes_anterior,indice_tendencia_ciclo,var_tendencia_mes_anterior
0,01/02/2004,90.186179,0.000000,98.182867,-0.255845,96.648290,0.621850
1,01/03/2004,101.883298,0.000000,97.784388,-0.405854,97.318642,0.693600
2,01/04/2004,102.567430,0.000000,95.129053,-2.715500,98.015884,0.716453
3,01/05/2004,109.877504,0.000000,96.891162,1.852336,98.740023,0.738798
4,01/06/2004,105.662406,0.000000,98.650311,1.815592,99.489725,0.759268
...,...,...,...,...,...,...,...
252,01/02/2025,141.160093,5.629332,153.378984,0.724647,151.413009,0.452763
253,01/03/2025,150.380892,5.479625,150.575664,-1.827708,152.007691,0.392755
254,01/04/2025,159.115637,8.046957,152.735548,1.434418,152.524114,0.339735
255,01/05/2025,165.068610,5.289033,152.440207,-0.193368,152.976638,0.296690


In [8]:
import plotly.graph_objects as go
import pandas as pd

# Ensure the 'fecha' column is in datetime format for plotting
df_emae['fecha_dt'] = pd.to_datetime(df_emae['fecha'], format='%d/%m/%Y')

# Create the Plotly figure
fig = go.Figure()

# Add the trace for 'indice_desestacionalizado' (line)
fig.add_trace(go.Scatter(
    x=df_emae['fecha_dt'],
    y=df_emae['indice_desestacionalizado'],
    mode='lines',
    name='EMAE (Índice Desestacionalizado)',
    yaxis='y1', # Associate with the primary Y axis
    line=dict(color='green') # Set line color
))

# Add the trace for 'var_respecto_mes_anterior' (bars)
fig.add_trace(go.Bar(
    x=df_emae['fecha_dt'],
    y=df_emae['var_respecto_mes_anterior'],
    name='EMAE (Variación Mensual)',
    yaxis='y2', # Associate with the secondary Y axis
    marker=dict(color='blue', opacity=0.7) # Set bar color and opacity
))


# Configure axes and layout
fig.update_layout(
    title=dict(
        text='EMAE (Índice Desestacionalizado y Variación Mensual) - Interactivo',
        x=0.5,
        xanchor='center'
    ),
    xaxis=dict(
        title='Fecha',
        titlefont=dict(color='black'),
        tickfont=dict(color='black'),
        showgrid=False
    ),
    yaxis=dict(
        title='Índice Desestacionalizado (Base 2004=100)',
        titlefont=dict(color='black'),
        tickfont=dict(color='black'),
        showgrid=False
    ),
    yaxis2=dict(
        title='Variación Mensual (%)',
        titlefont=dict(color='black'),
        tickfont=dict(color='black'),
        overlaying='y', # Superimpose on the primary Y axis
        side='right',    # Place on the right side
        showgrid=False # Eliminate grid for the secondary Y axis
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    hovermode='x unified',
    plot_bgcolor='white',
    font=dict(family='Arial')
)

# Show the interactive plot
fig.show()

# Drop the temporary datetime column
df_emae = df_emae.drop('fecha_dt', axis=1)

In [20]:
# Dashboard Económico Argentina - Versión Jupyter/Colab
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
from io import BytesIO
from datetime import datetime, timedelta
import numpy as np
import yfinance as yf
import calendar
from IPython.display import display, HTML
import ipywidgets as widgets
from IPython.display import clear_output

print("🇦🇷 Dashboard Económico Argentina")
print("=" * 50)

# === FUNCIONES DE EXTRACCIÓN DE DATOS ===

def obtener_datos_indec_completo():
    """Lee las 3 hojas principales del Excel del INDEC"""
    hoy = datetime.now()
    mes_actual_str = str(hoy.month).zfill(2)
    url = f"https://www.indec.gob.ar/ftp/cuadros/economia/sh_ipc_{mes_actual_str}_25.xls"

    try:
        print(f"📥 Descargando datos de inflación...")
        response = requests.get(url, timeout=30)
        response.raise_for_status()

        hojas_datos = {}
        nombres_hojas = [
            'Variación mensual IPC Nacional',
            'Var. interanual IPC Nacional',
            'Índices IPC Cobertura Nacional'
        ]

        for i, nombre in enumerate(nombres_hojas):
            try:
                df = pd.read_excel(BytesIO(response.content), sheet_name=i, engine='xlrd')
                hojas_datos[nombre] = df
            except Exception as e:
                print(f"⚠️ Error leyendo hoja {i}: {e}")

        print("✅ Datos de inflación descargados")
        return hojas_datos
    except Exception as e:
        print(f"❌ Error descargando datos de inflación: {e}")
        return {}

def extraer_nivel_general_de_hoja(df, nombre_hoja):
    """Extrae datos de 'Nivel general' de cualquier hoja"""
    if df.empty:
        return pd.DataFrame()

    # Buscar fila con fechas
    fila_fechas = None
    for i, fila in df.iterrows():
        fechas_count = sum(1 for val in fila.iloc[1:] if isinstance(val, datetime))
        if fechas_count > 5:
            fila_fechas = i
            break

    # Buscar fila "Nivel general"
    fila_nivel_general = None
    for i, fila in df.iterrows():
        if pd.notna(fila.iloc[0]) and str(fila.iloc[0]).strip() == 'Nivel general':
            fila_nivel_general = i
            break

    if fila_fechas is None or fila_nivel_general is None:
        return pd.DataFrame()

    # Extraer fechas y valores
    fechas = df.iloc[fila_fechas].values[1:]
    valores = df.iloc[fila_nivel_general].values[1:]

    # Crear DataFrame
    datos = []
    for fecha, valor in zip(fechas, valores):
        if pd.notna(fecha) and isinstance(fecha, datetime):
            if pd.notna(valor):
                try:
                    datos.append({
                        'fecha': fecha,
                        'valor': float(valor)
                    })
                except ValueError:
                    pass

    if datos:
        df_resultado = pd.DataFrame(datos).sort_values('fecha').reset_index(drop=True)
        return df_resultado

    return pd.DataFrame()

def combinar_datos_inflacion(hojas_datos):
    """Combina datos de las 3 hojas en un DataFrame completo"""
    datos_combinados = {}

    # Hoja 0: Variación mensual
    if 'Variación mensual IPC Nacional' in hojas_datos:
        df_mensual = extraer_nivel_general_de_hoja(
            hojas_datos['Variación mensual IPC Nacional'],
            'Variación mensual'
        )
        if not df_mensual.empty:
            datos_combinados['variacion_mensual'] = df_mensual.set_index('fecha')['valor']

    # Hoja 1: Variación interanual
    if 'Var. interanual IPC Nacional' in hojas_datos:
        df_interanual = extraer_nivel_general_de_hoja(
            hojas_datos['Var. interanual IPC Nacional'],
            'Variación interanual'
        )
        if not df_interanual.empty:
            datos_combinados['variacion_interanual'] = df_interanual.set_index('fecha')['valor']

    # Hoja 2: Índices IPC
    if 'Índices IPC Cobertura Nacional' in hojas_datos:
        df_indices = extraer_nivel_general_de_hoja(
            hojas_datos['Índices IPC Cobertura Nacional'],
            'Índices IPC'
        )
        if not df_indices.empty:
            datos_combinados['ipc_indice'] = df_indices.set_index('fecha')['valor']

    if not datos_combinados:
        return pd.DataFrame()

    # Combinar todos los datos
    df_final = pd.DataFrame(datos_combinados)
    df_final = df_final.reset_index()
    df_final['fecha'] = pd.to_datetime(df_final['fecha']).dt.tz_localize(None)
    df_final = df_final.sort_values('fecha').reset_index(drop=True)

    return df_final

def obtener_datos_inflacion():
    """Función principal que obtiene datos completos de inflación"""
    hojas_datos = obtener_datos_indec_completo()
    if not hojas_datos:
        return None

    df_completo = combinar_datos_inflacion(hojas_datos)
    if df_completo.empty:
        return None

    return df_completo

def obtener_datos_dolar():
    """Obtiene datos del tipo de cambio USD/ARS"""
    try:
        print("📥 Descargando datos del dólar...")
        dolar_argentina = yf.Ticker("ARS=X")
        historial = dolar_argentina.history(period="1y")

        if historial.empty:
            return None

        # Normalizar timezone
        historial.index = pd.to_datetime(historial.index).tz_localize(None)
        historial = historial.reset_index()
        historial = historial.rename(columns={'Date': 'fecha', 'Close': 'usd_ars'})
        historial['fecha'] = pd.to_datetime(historial['fecha']).dt.tz_localize(None)

        print("✅ Datos del dólar descargados")
        return historial[['fecha', 'usd_ars']]

    except Exception as e:
        print(f"❌ Error obteniendo datos del dólar: {e}")
        return None

def descargar_datos_emae():
    """Descarga y limpia los datos del EMAE desde el sitio del INDEC"""
    url = "https://www.indec.gob.ar/ftp/cuadros/economia/sh_emae_mensual_base2004.xls"

    try:
        print("📥 Descargando datos del EMAE...")
        response = requests.get(url, timeout=30)
        response.raise_for_status()

        excel_data = pd.read_excel(BytesIO(response.content), engine='xlrd', header=None)
        datos_procesados = []

        # Mapeo de nombres de meses
        meses = {
            'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4,
            'mayo': 5, 'junio': 6, 'julio': 7, 'agosto': 8,
            'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
        }

        año_actual = None

        # Procesar desde la fila 6
        for idx in range(5, len(excel_data)):
            row = excel_data.iloc[idx]
            año_val = row.iloc[0]
            mes_val = row.iloc[1]

            # Actualizar año actual si está disponible
            if pd.notna(año_val):
                año_str = str(año_val).strip()
                if año_str.isdigit() and len(año_str) == 4:
                    año_actual = int(año_str)

            # Procesar solo si tenemos año y mes válidos
            if año_actual is not None and pd.notna(mes_val):
                mes_nombre = str(mes_val).strip().lower()
                if mes_nombre in meses:
                    mes_num = meses[mes_nombre]

                    # Filtrar desde febrero 2004
                    if año_actual < 2004 or (año_actual == 2004 and mes_num < 2):
                        continue

                    fecha = f"01/{mes_num:02d}/{año_actual}"

                    # Extraer valores
                    valores = []
                    for col_idx in range(2, 8):
                        try:
                            if col_idx < len(row):
                                val = row.iloc[col_idx]
                                if pd.notna(val):
                                    val_str = str(val).replace(',', '.')
                                    val_clean = ''.join(c for c in val_str if c.isdigit() or c in '.-')
                                    if val_clean and val_clean.replace('.', '').replace('-', '').isdigit():
                                        valores.append(float(val_clean))
                                    else:
                                        valores.append(np.nan)
                                else:
                                    valores.append(np.nan)
                            else:
                                valores.append(np.nan)
                        except:
                            valores.append(np.nan)

                    # Crear registro
                    registro = {
                        'fecha': fecha,
                        'indice_serie_original': valores[0] if len(valores) > 0 else np.nan,
                        'var_respecto_año_anterior': valores[1] if len(valores) > 1 else np.nan,
                        'indice_desestacionalizado': valores[2] if len(valores) > 2 else np.nan,
                        'var_respecto_mes_anterior': valores[3] if len(valores) > 3 else np.nan,
                        'indice_tendencia_ciclo': valores[4] if len(valores) > 4 else np.nan,
                        'var_tendencia_mes_anterior': valores[5] if len(valores) > 5 else np.nan
                    }

                    datos_procesados.append(registro)

        # Crear DataFrame final
        if datos_procesados:
            df_limpio = pd.DataFrame(datos_procesados)
            df_limpio['fecha'] = pd.to_datetime(df_limpio['fecha'], format='%d/%m/%Y')
            df_limpio = df_limpio.sort_values('fecha').reset_index(drop=True)
            print("✅ Datos del EMAE descargados")
            return df_limpio
        else:
            return None

    except Exception as e:
        print(f"❌ Error al descargar datos del EMAE: {e}")
        return None

# === CARGAR DATOS ===
print("\n🔄 Cargando datos económicos...")
print("-" * 30)

# Cargar datos
df_inflacion = obtener_datos_inflacion()
df_dolar = obtener_datos_dolar()
df_emae = descargar_datos_emae()

# Verificar datos disponibles
datos_disponibles = []
if df_inflacion is not None and not df_inflacion.empty:
    datos_disponibles.append("Inflación")
if df_dolar is not None and not df_dolar.empty:
    datos_disponibles.append("Tipo de Cambio")
if df_emae is not None and not df_emae.empty:
    datos_disponibles.append("EMAE")

print(f"\n✅ Datos cargados exitosamente: {', '.join(datos_disponibles)}")

# === MOSTRAR MÉTRICAS ===
print("\n📊 INDICADORES PRINCIPALES")
print("=" * 50)

if df_inflacion is not None and not df_inflacion.empty:
    if 'variacion_mensual' in df_inflacion.columns:
        last_inf = df_inflacion['variacion_mensual'].dropna().iloc[-1]
        print(f"🔥 Inflación Mensual: {last_inf:.2f}%")

    if 'variacion_interanual' in df_inflacion.columns:
        last_inf_anual = df_inflacion['variacion_interanual'].dropna().iloc[-1]
        print(f"📈 Inflación Anual: {last_inf_anual:.1f}%")

if df_dolar is not None and not df_dolar.empty:
    last_dolar = df_dolar['usd_ars'].iloc[-1]
    if len(df_dolar) > 1:
        prev_dolar = df_dolar['usd_ars'].iloc[-2]
        delta_dolar = ((last_dolar - prev_dolar) / prev_dolar) * 100
        print(f"💵 USD/ARS: ${last_dolar:.0f} ({delta_dolar:+.2f}%)")
    else:
        print(f"💵 USD/ARS: ${last_dolar:.0f}")

if df_emae is not None and not df_emae.empty:
    if 'indice_desestacionalizado' in df_emae.columns:
        last_emae = df_emae['indice_desestacionalizado'].dropna().iloc[-1]
        if len(df_emae) > 1:
            prev_emae = df_emae['indice_desestacionalizado'].dropna().iloc[-2]
            delta_emae = ((last_emae - prev_emae) / prev_emae) * 100
            print(f"📊 EMAE: {last_emae:.1f} ({delta_emae:+.2f}%)")
        else:
            print(f"📊 EMAE: {last_emae:.1f}")

print("\n📈 GRÁFICOS INTERACTIVOS")
print("=" * 50)

# === GRÁFICOS ===

# 1. Inflación
if df_inflacion is not None and not df_inflacion.empty:
    print("\n🔥 Análisis de Inflación")

    # Gráfico combinado de inflación
    fig_inf = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Inflación Mensual (%)', 'Inflación Interanual (%)'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}]]
    )

    if 'variacion_mensual' in df_inflacion.columns:
        fig_inf.add_trace(
            go.Bar(x=df_inflacion['fecha'], y=df_inflacion['variacion_mensual'],
                   name='Mensual', marker_color='orange'),
            row=1, col=1
        )

    if 'variacion_interanual' in df_inflacion.columns:
        fig_inf.add_trace(
            go.Scatter(x=df_inflacion['fecha'], y=df_inflacion['variacion_interanual'],
                      mode='lines', name='Anual', line=dict(color='red', width=2)),
            row=1, col=2
        )

    fig_inf.update_layout(height=400, showlegend=False)
    fig_inf.show()

# 2. Dólar
if df_dolar is not None and not df_dolar.empty:
    print("\n💵 Tipo de Cambio USD/ARS")

    fig_dolar = go.Figure()
    fig_dolar.add_trace(go.Scatter(
        x=df_dolar['fecha'], y=df_dolar['usd_ars'],
        mode='lines', name='USD/ARS',
        line=dict(color='green', width=2)
    ))

    fig_dolar.update_layout(
        title="Evolución del Dólar (USD/ARS)",
        xaxis_title="Fecha",
        yaxis_title="ARS por USD",
        height=400
    )
    fig_dolar.show()

# 3. EMAE
if df_emae is not None and not df_emae.empty and 'indice_desestacionalizado' in df_emae.columns:
    print("\n📊 EMAE - Actividad Económica")

    fig_emae = go.Figure()
    fig_emae.add_trace(go.Scatter(
        x=df_emae['fecha'], y=df_emae['indice_desestacionalizado'],
        mode='lines', name='EMAE',
        line=dict(color='blue', width=2)
    ))

    fig_emae.update_layout(
        title="EMAE (Índice Desestacionalizado)",
        xaxis_title="Fecha",
        yaxis_title="Índice (Base 2004=100)",
        height=400
    )
    fig_emae.show()

# 4. Gráfico Combinado
print("\n📈 Vista Integrada - Variables Normalizadas")

fig_combined = go.Figure()

if df_inflacion is not None and not df_inflacion.empty and 'variacion_mensual' in df_inflacion.columns:
    inf_vals = df_inflacion['variacion_mensual'].dropna()
    if len(inf_vals) > 0:
        inf_norm = (inf_vals - inf_vals.min()) / (inf_vals.max() - inf_vals.min())
        fig_combined.add_trace(go.Scatter(
            x=df_inflacion.loc[inf_vals.index, 'fecha'], y=inf_norm,
            mode='lines', name='Inflación (norm.)',
            line=dict(color='orange', width=2)
        ))

if df_dolar is not None and not df_dolar.empty:
    dolar_vals = df_dolar['usd_ars'].dropna()
    if len(dolar_vals) > 0:
        dolar_norm = (dolar_vals - dolar_vals.min()) / (dolar_vals.max() - dolar_vals.min())
        fig_combined.add_trace(go.Scatter(
            x=df_dolar.loc[dolar_vals.index, 'fecha'], y=dolar_norm,
            mode='lines', name='USD/ARS (norm.)',
            line=dict(color='green', width=2)
        ))

if df_emae is not None and not df_emae.empty and 'indice_desestacionalizado' in df_emae.columns:
    emae_vals = df_emae['indice_desestacionalizado'].dropna()
    if len(emae_vals) > 0:
        emae_norm = (emae_vals - emae_vals.min()) / (emae_vals.max() - emae_vals.min())
        fig_combined.add_trace(go.Scatter(
            x=df_emae.loc[emae_vals.index, 'fecha'], y=emae_norm,
            mode='lines', name='EMAE (norm.)',
            line=dict(color='blue', width=2)
        ))

fig_combined.update_layout(
    title="Comparación de Variables Económicas (Normalizadas 0-1)",
    xaxis_title="Fecha",
    yaxis_title="Valor Normalizado",
    height=500,
    hovermode='x unified'
)
fig_combined.show()

# === TABLA DE DATOS RECIENTES ===
print("\n📋 Datos Recientes")
if df_inflacion is not None and not df_inflacion.empty:
    print("\n📊 Últimos 10 registros de inflación:")
    tabla_inflacion = df_inflacion.tail(10).copy()
    tabla_inflacion['fecha'] = tabla_inflacion['fecha'].dt.strftime('%Y-%m')

    # Formatear columnas numéricas
    for col in ['variacion_mensual', 'variacion_interanual', 'ipc_indice']:
        if col in tabla_inflacion.columns:
            tabla_inflacion[col] = tabla_inflacion[col].round(2)

    display(tabla_inflacion[['fecha', 'variacion_mensual', 'variacion_interanual']].head())

print("\n" + "="*50)
print("🔍 INFORMACIÓN:")
print("• Inflación: INDEC (Instituto Nacional de Estadística y Censos)")
print("• Tipo de Cambio: Yahoo Finance (USD/ARS)")
print("• EMAE: INDEC (Estimador Mensual de Actividad Económica)")
print("• Gráficos interactivos: Pasa el mouse sobre los gráficos para más detalles")
print("="*50)

🇦🇷 Dashboard Económico Argentina

🔄 Cargando datos económicos...
------------------------------
📥 Descargando datos de inflación...
✅ Datos de inflación descargados
📥 Descargando datos del dólar...
✅ Datos del dólar descargados
📥 Descargando datos del EMAE...
✅ Datos del EMAE descargados

✅ Datos cargados exitosamente: Inflación, Tipo de Cambio, EMAE

📊 INDICADORES PRINCIPALES
🔥 Inflación Mensual: 1.90%
📈 Inflación Anual: 33.6%
💵 USD/ARS: $1465 (+0.95%)
📊 EMAE: 151.4 (-0.70%)

📈 GRÁFICOS INTERACTIVOS

🔥 Análisis de Inflación



💵 Tipo de Cambio USD/ARS



📊 EMAE - Actividad Económica



📈 Vista Integrada - Variables Normalizadas



📋 Datos Recientes

📊 Últimos 10 registros de inflación:


,fecha,variacion_mensual,variacion_interanual
95,2024-11,2.4,166.0
96,2024-12,2.7,117.8
97,2025-01,2.2,84.5
98,2025-02,2.4,66.9
99,2025-03,3.7,55.9



🔍 INFORMACIÓN:
• Inflación: INDEC (Instituto Nacional de Estadística y Censos)
• Tipo de Cambio: Yahoo Finance (USD/ARS)
• EMAE: INDEC (Estimador Mensual de Actividad Económica)
• Gráficos interactivos: Pasa el mouse sobre los gráficos para más detalles
